In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sympy as sp
from collections import OrderedDict
from pympc.algebra import clean_matrix

In [3]:
def symVec(length, name):
    return sp.Matrix([[sp.symbols(name + '_' + str(i))] for i in range(length)])

In [4]:
p = {
    'mb': 1.,
    'ml': .1,
    'ib': 1.,
    'mu': .5,
    'g': np.array([[0.], [-10.]]),
    'h': .1,
}

q_bounds = OrderedDict([
    ('bl', {'min': np.array([[-.2],[-.1]]), 'max': np.array([[.2],[.1]])}),
    ('ba', {'min': np.array([[-.1]]), 'max': np.array([[.1]])}),
    ('lf', {'min': np.array([[.0],[-.7]]), 'max': np.array([[.4],[-.3]])}),
    ('rf', {'min': np.array([[-.4],[-.7]]), 'max': np.array([[0.],[-.3]])}),
    ('lh', {'min': np.array([[.2],[-.2]]), 'max': np.array([[.4],[.2]])}),
    ('rh', {'min': np.array([[-.4],[-.2]]), 'max': np.array([[-.2],[.2]])}),
    ])

v_bounds = OrderedDict([
    ('bl', {'min': np.array([[-2.],[-2.]]), 'max': np.array([[2.],[2.]])}),
    ('ba', {'min': np.array([[-.5]]), 'max': np.array([[.5]])}),
    ('lf', {'min': -np.ones((2,1)), 'max': np.ones((2,1))}),
    ('rf', {'min': -np.ones((2,1)), 'max': np.ones((2,1))}),
    ('lh', {'min': -2*np.ones((2,1)), 'max': 2*np.ones((2,1))}),
    ('rh', {'min': -2*np.ones((2,1)), 'max': 2*np.ones((2,1))}),
    ])

In [5]:
class Limb():
    '''
    Domains of the limbs are polyhedrons in the form {q | A q <= b}.
    The contact surface coincides with the first inequality, i.e. A_0 q = b_0.
    '''
    def __init__(self, A, b, q_nom):
        self.A = A
        self.b = b
        self.q_nom = q_nom
        self.n = - A[0,:].reshape(A.shape[1], 1)
        self.d = - b[0,0]
        rotation = np.array([[0., -1.],[1., 0.]])
        self.t = rotation.dot(self.n)
        self.R = np.hstack((self.n, self.t))

# left hand
A_lh = np.array([[1., 0.],[0., -1.]])
b_lh = np.array([[.5],[.5]])
q_lh = np.array([[.3], [.0]])
lh = Limb(A_lh, b_lh, q_lh)
        
# right hand
A_rh = np.array([[-1., 0.],[0., -1.]])
b_rh = np.array([[.5],[.5]])
q_rh = np.array([[-.3], [.0]])
rh = Limb(A_rh, b_rh, q_rh)

# left foot
A_lf = np.array([[0., -1.],[-1., 0.]])
b_lf = np.array([[.5],[.5]])
q_lf = np.array([[.2], [-.5]])
lf = Limb(A_lf, b_lf, q_lf)

# right foot
A_rf = np.array([[0., -1.],[-1., 0.]])
b_rf = np.array([[.5],[.5]])
q_rf = np.array([[-.2], [-.5]])
rf = Limb(A_rf, b_rf, q_rf)

# list of limbs
limbs = {'lf': lf, 'rf': rf, 'rh': rh, 'lh': lh}

In [6]:
q = OrderedDict([
    ('bl', symVec(2, 'qbl')),
    ('ba', symVec(1, 'qba'))
])
for label, limb in limbs.items():
    q[label] = symVec(2, 'q'+label)
q_vec = sp.Matrix([q_i for q_i in q.values()])

v = OrderedDict([
    ('bl', symVec(2, 'vbl')),
    ('ba', symVec(1, 'vba'))
])
for label in limbs.keys():
    v[label] = symVec(2, 'v'+label)
v_vec = sp.Matrix([v_i for v_i in v.values()])

In [7]:
q_vec_nom = sp.Matrix([[0.]]*3)
for limb in limbs.values():
    q_vec_nom = q_vec_nom.col_join(sp.Matrix(limb.q_nom))
    
v_vec_nom = sp.Matrix([[0.]]*v_vec.shape[0])

In [8]:
def rotation(th):
    return sp.Matrix([[sp.cos(th), -sp.sin(th)],[sp.sin(th), sp.cos(th)]])

positions = OrderedDict([('bl', q['bl']), ('ba', q['ba'])])
for label in limbs.keys():
    positions[label] = q['bl'] + rotation(q['ba'][0,0])*q[label]
    
velocities = OrderedDict([('bl', v['bl']), ('ba', v['ba'])])
for label in limbs.keys():
    velocities[label] = positions[label].jacobian(q_vec)*v_vec

In [9]:
kinetic_energy = .5*p['mb']*velocities['bl'].T*(velocities['bl'])
kinetic_energy += .5*p['ib']*velocities['ba']*(velocities['ba'])
for label in limbs.keys():
    kinetic_energy += .5*p['ml']*velocities[label].T*velocities[label]

potential_energy = -p['mb']*p['g'].T*positions['bl']
for label in limbs.keys():
    potential_energy += -p['ml']*p['g'].T*positions[label]

lagrangian = kinetic_energy - potential_energy

In [10]:
u = OrderedDict()
generalized_forces = np.zeros(q_vec.shape)
for label, limb in limbs.items():
    u[label] = symVec(2, 'u'+label)
    displacement = positions[label] - positions['bl']
    displacement = (positions[label] - positions['bl']).jacobian(q_vec)
    generalized_forces = generalized_forces + displacement.T*u[label]
u_vec = sp.Matrix([u_i for u_i in u.values()])
wl = p['g']*p['ml']
u_vec_nom = sp.Matrix(list(-wl)*len(limbs))

f = OrderedDict()
for label, limb in limbs.items():
    f[label] = symVec(2, 'f'+label)
    displacement = positions[label].jacobian(q_vec)
    R = sp.Matrix(limb.R)
    generalized_forces = generalized_forces + displacement.T*R*f[label]
f_vec = sp.Matrix([f_i for f_i in f.values()])
f_vec_nom = sp.Matrix([[0.]]*f_vec.shape[0])

Equations of motion in the form

$M(q) \dot v + c(q, v) = J_f(q) f + J_u(q) u$

In [25]:
M = lagrangian.jacobian(v_vec).T.jacobian(v_vec)
c = lagrangian.jacobian(v_vec).T.jacobian(q_vec)*v_vec - lagrangian.jacobian(q_vec).T
Ju = generalized_forces.jacobian(u_vec)
Jf = generalized_forces.jacobian(f_vec)

#M = sp.simplify(M)
#c = sp.simplify(c)
#Ju = sp.simplify(Ju)
#Jf = sp.simplify(Jf)
for i in range(Jf.shape[0]):
    print Jf[i,:]

Matrix([[0, -1.00000000000000, 0, -1.00000000000000, 1.00000000000000, 0, -1.00000000000000, 0]])
Matrix([[1.00000000000000, 0, 1.00000000000000, 0, 0, 1.00000000000000, 0, -1.00000000000000]])
Matrix([[1.0*qlf_0*cos(qba_0) - 1.0*qlf_1*sin(qba_0), 1.0*qlf_0*sin(qba_0) + 1.0*qlf_1*cos(qba_0), 1.0*qrf_0*cos(qba_0) - 1.0*qrf_1*sin(qba_0), 1.0*qrf_0*sin(qba_0) + 1.0*qrf_1*cos(qba_0), -1.0*qrh_0*sin(qba_0) - 1.0*qrh_1*cos(qba_0), 1.0*qrh_0*cos(qba_0) - 1.0*qrh_1*sin(qba_0), 1.0*qlh_0*sin(qba_0) + 1.0*qlh_1*cos(qba_0), -1.0*qlh_0*cos(qba_0) + 1.0*qlh_1*sin(qba_0)]])
Matrix([[1.0*sin(qba_0), -1.0*cos(qba_0), 0, 0, 0, 0, 0, 0]])
Matrix([[1.0*cos(qba_0), 1.0*sin(qba_0), 0, 0, 0, 0, 0, 0]])
Matrix([[0, 0, 1.0*sin(qba_0), -1.0*cos(qba_0), 0, 0, 0, 0]])
Matrix([[0, 0, 1.0*cos(qba_0), 1.0*sin(qba_0), 0, 0, 0, 0]])
Matrix([[0, 0, 0, 0, 1.0*cos(qba_0), 1.0*sin(qba_0), 0, 0]])
Matrix([[0, 0, 0, 0, -1.0*sin(qba_0), 1.0*cos(qba_0), 0, 0]])
Matrix([[0, 0, 0, 0, 0, 0, -1.0*cos(qba_0), -1.0*sin(qba_0)]])
M

In [12]:
def matrix_subs(M, x_list, x_list_subs):
    M_subs = M
    for j, x in enumerate(x_list):
        for i in range(x.shape[0]):
            M_subs = M_subs.subs(x[i], x_list_subs[j][i])
    return M_subs

Linearization:

$M(\bar q) \dot v + c(\bar q, \bar v) +
\frac{\partial c}{\partial q}(\bar q, \bar v) (q - \bar q) +
\frac{\partial c}{\partial v}(\bar q, \bar v) (v - \bar v) =
J_u(\bar q) u +
\frac{\partial J_u u}{\partial q} (\bar q, \bar u) (q - \bar q) + 
J_f(\bar q) f +
\frac{\partial J_f f}{\partial q} (\bar q, \bar f) (q - \bar q)$

In [13]:
M_nom = matrix_subs(M, [q_vec], [q_vec_nom])
c_nom = matrix_subs(c, [q_vec, v_vec], [q_vec_nom, v_vec_nom])
c_q = c.jacobian(q_vec)
c_q_nom = matrix_subs(c_q, [q_vec, v_vec], [q_vec_nom, v_vec_nom])
c_v = c.jacobian(v_vec)
c_v_nom = matrix_subs(c_v, [q_vec, v_vec], [q_vec_nom, v_vec_nom])
Ju_nom = matrix_subs(Ju, [q_vec], [q_vec_nom])
Ju_u_q = (Ju*u_vec).jacobian(q_vec)
Ju_u_q_nom = matrix_subs(Ju_u_q, [q_vec, u_vec], [q_vec_nom, u_vec_nom])
Jf_nom = matrix_subs(Jf, [q_vec], [q_vec_nom])
Jf_f_q = (Jf*f_vec).jacobian(q_vec)
Jf_f_q_nom = matrix_subs(Jf_f_q, [q_vec, f_vec], [q_vec_nom, f_vec_nom])

State space representation

$
\begin{bmatrix}
\dot q \\ \dot v
\end{bmatrix}
=
\begin{bmatrix}
0 & I \\
M(\bar q)^{-1} \left(
- \frac{\partial c}{\partial q}(\bar q, \bar v)
+ \frac{\partial J_u u}{\partial q} (\bar q, \bar u)
+ \frac{\partial J_f f}{\partial q} (\bar q, \bar f)
\right) &
-M(\bar q)^{-1}
\frac{\partial c}{\partial v}(\bar q, \bar v)
\end{bmatrix}
\begin{bmatrix}
q - \bar q \\ v - \bar v
\end{bmatrix}
+
\begin{bmatrix}
0 \\ M(\bar q)^{-1} J_u(\bar q) 
\end{bmatrix}
(u - \bar u)
+
\begin{bmatrix}
0 \\ M(\bar q)^{-1} J_f(\bar q) 
\end{bmatrix}
(f - \bar f)
+
\begin{bmatrix}
0
\\
M(\bar q)^{-1} \left(
-c(\bar q, \bar v)
+ J_u(\bar q) \bar u
+ J_f(\bar q) \bar f
\right)
\end{bmatrix}
$


In [14]:
A00 = sp.zeros(q_vec.shape[0])
A01 = sp.eye(q_vec.shape[0])
A10 = M_nom.inv()*(-c_q_nom + Ju_u_q_nom + Jf_f_q_nom)
A11 = - M_nom.inv()*c_v_nom
A = sp.Matrix([A00.row_join(A01), A10.row_join(A11)])
Bu0 = sp.zeros(q_vec.shape[0], u_vec.shape[0])
Bu1 = M_nom.inv()*Ju_nom
Bu = sp.Matrix([Bu0, Bu1])
Bf0 = sp.zeros(q_vec.shape[0], f_vec.shape[0])
Bf1 = M_nom.inv()*Jf_nom
Bf = sp.Matrix([Bf0, Bf1])
c0 = sp.zeros(q_vec.shape[0], 1)
c1 = M_nom.inv()*(- c_nom + Ju_nom*u_vec_nom + Jf_nom*f_vec_nom)
c = sp.Matrix([c0, c1])

In [15]:
def matrix2numpy(m):
    a = np.empty(m.shape)
    for i in range(m.rows):
        for j in range(m.cols):
            a[i, j] = m[i, j]
    return clean_matrix(a)

A = matrix2numpy(A)
Bu = matrix2numpy(Bu)
Bf = matrix2numpy(Bf)
c = matrix2numpy(c)

Solve the complementarity

In [16]:
active_contacts = ['lf','rf']
for contact in active_contacts:
    l = limbs[contact]
    q_limb = l['d']*l['n'] 


SyntaxError: invalid syntax (<ipython-input-16-5f591d8b65fe>, line 4)

In [17]:
Bf

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0., -10.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 10.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0., -10.,   0.,   0.,   0.,   0.],
       [  0.,   0.,  10.,   0.,

In [18]:
q

OrderedDict([('bl', Matrix([
              [qbl_0],
              [qbl_1]])), ('ba', Matrix([[qba_0]])), ('lf', Matrix([
              [qlf_0],
              [qlf_1]])), ('rf', Matrix([
              [qrf_0],
              [qrf_1]])), ('rh', Matrix([
              [qrh_0],
              [qrh_1]])), ('lh', Matrix([
              [qlh_0],
              [qlh_1]]))])

In [ ]:
potentia